In [1]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [1]:
# ##uncomment this to use code in Google Colab
# !pip install fastchat
# !pip install wget
# !pip install evaluate
# !pip install sentence_transformers
# !pip install bert_score
# !pip install datasets

In [2]:
# ##uncomment this to use code in Google Colab
# from IPython.display import clear_output
# !git clone https://github.com/alfekka/lm-polygraph.git -b new_api
# %cd lm-polygraph
# %pip install .
# %cd src
# %pip install transformers rouge-score datasets openai
# !curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
# !apt-get install -y nodejs
# %cd lm_polygraph/app
# !npm install
# %cd ../../
# %cd /content/lm-polygraph/src
# %load_ext autoreload
# %autoreload 2

In [8]:
# ##uncomment this to use code in Google Colab
# %cd ..
# !pwd
# %cd /content/lm-polygraph/src

/content/lm-polygraph
/content/lm-polygraph
/content/lm-polygraph/src


In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
import os
from lm_polygraph.stat_calculators import *
from lm_polygraph.utils.model import WhiteboxModel
from lm_polygraph.utils.openai_chat import OpenAIChat
from transformers import AutoTokenizer, AutoModelForCausalLM
from lm_polygraph.model_adapters import WhiteBoxModelBasic
# from lm_polygraph.model_adapters import WhiteBoxModel
from torch.utils.data import DataLoader
import json
import pandas as pd
from tqdm import tqdm
## Create a ClaimsExtractor object
calc_claim_extractor = ClaimsExtractor(OpenAIChat("gpt-4"))
## Set model to None, as we will not need it
model = None
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased", use_fast=True, trust_remote_code=True)
## Setting a dummy whitebox model object
model_adapter = WhiteBoxModelBasic(model, tokenizer)

In [12]:
from datasets import load_dataset
full = load_dataset("rvanova/person-bio-part")
russian_bios = pd.DataFrame(full['test'])

Generating test split:   0%|          | 0/70 [00:00<?, ? examples/s]

In [65]:
bios_with_claims = []
## Create a DataLoader object
for  index, row in tqdm(russian_bios.iterrows(), total=russian_bios.shape[0]):

    question = row['input_texts']
    bio = row['bio_ru']


    encoded = tokenizer([bio], padding=True, return_tensors="pt")
    deps = {"greedy_tokens": encoded.input_ids, "greedy_texts": [bio]}
    claims = calc_claim_extractor(deps, texts=[bio], model=model_adapter,language="ru")


    for x in claims['claims'][0]:
        bios_with_claims.append({'question': question, 'bio': bio, "claim": x.claim_text, "sentence": x.sentence, "tokens": x.aligned_tokens})


100%|██████████| 70/70 [1:22:52<00:00, 71.04s/it]


In [67]:
## Save the extracted claims

save_df = pd.DataFrame(bios_with_claims, )

save_df.to_csv("../workdir/russian_claims.csv", index=False)

In [68]:
len(save_df)

298

In [69]:
save_df['question'].unique().shape

(57,)